# Imports

In [18]:
import os
import warnings
warnings.filterwarnings("ignore")

from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import scipy as sp
import scipy.signal
import scipy.stats
from scipy.stats.mstats import gmean

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import GridSearchCV, LeaveOneOut, StratifiedKFold
from sklearn.svm import SVC

from utils import Dataset, variance_thresholding, standardize, mcc, calculate_metrics, calculate_metrics_statistics

In [2]:
# parameters for Welch's method for estimating power spectrum

NPERSEG = 24                    # length of segment
NOVERLAP = int(0.75 * NPERSEG)  # overlap of segments
NFFT = NPERSEG                  # length of FFT
WINDOW = "hann"                 # window function type

# parameters for saving data
PROCESSED_DATA_DIR = "processed_data24h"
DEPRESJON_PREFIX = "manual_depresjon24h"
PSYKOSE_PREFIX = "manual_psykose24h"

# Manual feature extraction

## Helper functions

In [3]:
def basic_data_cleaning(data: List[pd.DataFrame]) -> List[pd.DataFrame]:
    """
    Assumes DataFrames with "timestamp", "date" and "activity" columns.
    
    Performs cleaning operations:
    - assure format YYYY-MM-DD HH:MM:SS for "timestamp"
    - drop redundant "date" column
    - assure float32 format for "activity"
    
    :param data: list of DataFrames
    :returns: list of cleaned DataFrames
    """
    data = [df.copy() for df in data]  # create copy to avoid side effects
    
    for df in data:
        df["timestamp"] = pd.to_datetime(df["timestamp"],
                                         format="%Y-%m-%d %H:%M:%S")
        df.drop("date", axis=1, inplace=True)
        df["activity"] = df["activity"].astype(np.float32)
    
    return data


def get_day_part(df: pd.DataFrame, part: str) -> pd.DataFrame:
    """
    For given DataFrame with "timestamp" column returns only those rows that
    correspond to the chosen part of day.
    
    Parts are "day" and "night", defined as:
    - "day": [8:00, 21:00)
    - "night": [21:00, 8:00)
    
    :param df: DataFrame to select rows from
    :param part: part of day, either "day" or "night"
    :returns: DataFrame, subset of rows of df
    """
    if part == "day":
        df = df.loc[(df["timestamp"].dt.hour >= 8) &
                    (df["timestamp"].dt.hour < 21)]
    elif part == "night":
        df = df.loc[(df["timestamp"].dt.hour >= 21) |
                    (df["timestamp"].dt.hour < 8)]
    else:
        raise ValueError(f'Part should be "day" or "night", got "{part}"')
        
    return df


def fill_missing_activity(df: pd.DataFrame) -> pd.DataFrame:
    """
    Makes sure that "timestamp" column has minute resolution with no missing
    values from start to end and replaces all NaNs in "activity" column with
    mean average value.
    
    :param data: DataFrame with "timestamp" and "activity" columns
    :returns: cleaned DataFrame
    """
    df = df.copy()  # create copy to avoid side effects
    
    # resample to the basic frequency, i.e. minute; this will create NaNs for
    # any rows that may be missing
    df = df.resample("min", on="timestamp").mean()
    
    # recreate index and "timestamp" column
    df = df.reset_index()
    
    # fill any NaNs with mean activity value
    df["activity"] = df["activity"].fillna(df["activity"].mean())

    return df


def resample(df: pd.DataFrame, freq: str = "H") -> pd.DataFrame:
    """
    Resamples time series DataFrame with given frequency, aggregating each
    segment with a mean.

    :param df: DataFrame with "timestamp" and "activity" columns
    :param freq: resampling frequency passed to Pandas resample() function
    :returns: DataFrame with "timestamp" and "activity" columns
    """
    df = df.copy()  # create copy to avoid side effects
    
    # group with given frequency
    df = df.resample(freq, on="timestamp").mean()

    # recreate "timestamp" column
    df = df.reset_index()

    return df


def proportion_of_zeros(x: np.ndarray) -> float:
    """
    Calculates proportion of zeros in given array, i.e. number of zeros divided
    by length of array.
    
    :param x: 1D Numpy array
    :returns: proportion of zeros
    """
    # we may be dealing with floating numbers, we can't use direct comparison
    zeros_count = np.sum(np.isclose(x, 0))
    return zeros_count / len(x)


def power_spectral_density(df: pd.DataFrame) -> np.ndarray:
    """
    Calculates power spectral density (PSD) from "activity" column of a
    DataFrame.
    
    :param df: DataFrame with "activity" column
    :returns: 1D Numpy array with power spectral density
    """
    psd = scipy.signal.welch(
        x=df["activity"].values,
        fs=(1/24),
        nperseg=11,
        noverlap=10,
        nfft=NFFT,
        window=WINDOW,
        scaling="density"
    )[1]
    return psd


def spectral_flatness(df: pd.DataFrame) -> float:
    """
    Calculates spectral flatness of a signal, i.e. a geometric mean of the
    power spectrum divided by the arithmetic mean of the power spectrum.
    
    If some frequency bins in the power spectrum are close to zero, they are
    removed prior to calculation of spectral flatness to avoid calculation of
    log(0).
    
    :param df: DataFrame with "activity" column
    :returns: spectral flatness value
    """
    power_spectrum = scipy.signal.welch(
        df["activity"].values,
        fs=(1/24),
        nperseg=11,
        noverlap=10,
        nfft=NFFT,
        window=WINDOW,
        scaling="spectrum"
    )[1]
    
    non_zeros_mask = ~np.isclose(power_spectrum, 0)
    power_spectrum = power_spectrum[non_zeros_mask]
    
    return scipy.stats.gmean(power_spectrum) / power_spectrum.mean()

## Feature extraction

In [4]:
def extract_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in time domain.
    
    :param df_resampled: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = df["activity"].values
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X, ddof=1),  # apply Bessel's correction
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "proportion_of_zeros": proportion_of_zeros(X)
    }
    
    return pd.DataFrame([features])

In [5]:
def extract_frequency_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in frequency domain, i.e. calculated
    from its Power Spectral Density (PSD).
    
    :param df: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = power_spectral_density(df)
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X),
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "spectral_flatness": spectral_flatness(df)
    }
    
    return pd.DataFrame([features])

In [6]:
def extract_features_for_dataframes(dfs: List[pd.DataFrame], freq: str = "H") \
        -> Dict[str, pd.DataFrame]:
    """
    Calculates time and frequency features for given DataFrames. Uses given
    frequency for resampling.
    
    Calculates features separately for:
    - full 24hs
    - days: [8:00, 21:00)
    - nights: [21:00, 8:00)
    
    :param dfs: list of DataFrames to extract features from; each one has to
    have "timestamp" and "activity" columns
    :param freq: resampling frequency
    :returns: dictionary with keys "full_24h", "day" and "night", corresponding
    to features from given parts of day
    """
    full_dfs = basic_data_cleaning(dfs)
    full_dfs = [fill_missing_activity(df) for df in full_dfs]
    full_dfs = [resample(df, freq=freq) for df in full_dfs]
    
    night_dfs = [get_day_part(df, part="night") for df in full_dfs]
    day_dfs = [get_day_part(df, part="day") for df in full_dfs]

    datasets = {}
    
    for part, list_of_dfs in [("full_24h", full_dfs), ("night", night_dfs),
                              ("day", day_dfs)]:
        features = []
        for df in list_of_dfs:
            time_features = extract_time_features(df)
            freq_features = extract_frequency_features(df)

            merged_features = pd.merge(
                time_features,
                freq_features,
                left_index=True,
                right_index=True,
                suffixes=["_time", "_freq"]
            )
            features.append(merged_features)

        datasets[part] = pd.concat(features)
        datasets[part].reset_index(drop=True, inplace=True)
    
    return datasets



## Depresjon

In [7]:
dataset = Dataset(dirpath=os.path.join("data_24h", "depresjon"))
condition = dataset.condition
control = dataset.control

In [8]:
condition[0]

,timestamp,date,activity
0,2003-05-07 12:00:00,2003-05-07,0
1,2003-05-07 12:01:00,2003-05-07,143
2,2003-05-07 12:02:00,2003-05-07,0
3,2003-05-07 12:03:00,2003-05-07,20
4,2003-05-07 12:04:00,2003-05-07,166
...,...,...,...
1435,2003-05-08 11:55:00,2003-05-08,259
1436,2003-05-08 11:56:00,2003-05-08,190
1437,2003-05-08 11:57:00,2003-05-08,306
1438,2003-05-08 11:58:00,2003-05-08,91


In [9]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["full_24h"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    
    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [10]:
for part, df in datasets.items():
    filename = f"{DEPRESJON_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [11]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"depresjon_y.csv")
y.to_csv(filepath, header=False, index=False)

In [12]:
datasets["full_24h"].dropna(inplace=True)
datasets["full_24h"]

,minimum_time,maximum_time,mean_time,median_time,variance_time,kurtosis_time,skewness_time,coeff_of_var_time,iqr_time,trimmed_mean_time,...,mean_freq,median_freq,variance_freq,kurtosis_freq,skewness_freq,coeff_of_var_freq,iqr_freq,trimmed_mean_freq,entropy_freq,spectral_flatness
0,0.966667,395.649994,131.482635,105.699997,16655.847656,-0.920654,0.638463,0.960889,212.062506,120.149170,...,145556.015625,94128.164062,1.245808e+10,0.335413,1.309549,0.766824,68011.867188,131411.703125,3.339275,0.781018
1,2.083333,432.483337,153.253464,97.983337,20828.339844,-1.155962,0.531038,0.921882,247.795844,142.218353,...,225564.921875,66185.421875,7.144961e+10,-0.185189,1.162469,1.185027,291117.929688,191007.359375,2.800678,0.442222
2,0.583333,339.666656,124.177773,99.625000,12966.193359,-1.028185,0.526891,0.897678,184.895827,115.246666,...,337368.875000,233032.718750,9.313090e+10,-0.446068,0.971081,0.904569,379323.562500,306780.781250,3.145580,0.615705
3,0.916667,447.583344,128.460403,101.724998,14834.280273,0.110828,0.837585,0.928159,193.966663,114.235825,...,293284.031250,243482.046875,2.841528e+10,-1.273162,0.381854,0.574761,266578.562500,286987.468750,3.457804,0.826515
4,2.666667,368.250000,102.386108,59.950001,11585.225586,0.219186,1.074547,1.029128,136.641668,88.289169,...,75315.054688,50414.445312,2.197785e+09,0.339903,1.245119,0.622459,29237.988281,69791.164062,3.455590,0.847506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,0.000000,0.950000,0.039583,0.000000,0.037604,19.043478,4.587317,4.795832,0.000000,0.000000,...,2.289002,2.780218,5.765709e-01,0.321710,-1.308182,0.331727,0.825352,2.412409,3.599787,0.904552
1049,0.000000,0.816667,0.058333,0.000000,0.040242,8.400519,3.171385,3.366502,0.000000,0.000000,...,0.236217,0.231677,5.128690e-03,0.249235,-0.071112,0.303174,0.016871,0.235667,3.629814,0.945994
1050,0.000000,0.033333,0.001389,0.000000,0.000046,19.043478,4.587318,4.795832,0.000000,0.000000,...,0.000408,0.000386,1.543398e-08,-0.032443,0.335318,0.304669,0.000028,0.000403,3.632658,0.951204
1051,0.000000,79.416664,3.309028,0.000000,262.791931,19.043477,4.587317,4.795832,0.000000,0.000000,...,8990.534180,10806.850586,8.876594e+06,0.656679,-1.430759,0.331389,2489.757812,9508.233398,3.597398,0.898052


## Psykose

In [13]:
dataset = Dataset(dirpath=os.path.join("data_24h", "psykose"))
condition = dataset.condition
control = dataset.control

In [14]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["full_24h"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    
    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [15]:
for part, df in datasets.items():
    filename = f"{PSYKOSE_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [16]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"psykose_y.csv")
y.to_csv(filepath, header=False, index=False)
datasets["full_24h"].dropna(inplace=True)

# Classification

## Classifiers, parameters, constants

In [19]:
classifiers = {
    "LR": LogisticRegression(
        penalty="elasticnet",
        random_state=0,
        solver="saga",
        max_iter=500
    ),
    "SVM": SVC(
        kernel="rbf",
        cache_size=512
    ),
    "RF": RandomForestClassifier(
        n_estimators=200,
        criterion="entropy"
    ),
    "LGBM": LGBMClassifier(
        n_estimators=200,
        verbosity=-1,
        random_state=0
    ),
    "XGB": XGBClassifier(
        n_estimators=200,
        random_state=0 
    )
}


param_grids = {
    "LR": {
        "C": [0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 25, 50, 100, 500, 1000],
        "class_weight": ["balanced"],
        "l1_ratio": [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
                     0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
    },
    "SVM": {
        "class_weight": [None, "balanced"]
    },
    "RF": {
        "class_weight": [None, "balanced", "balanced_subsample"]
    },
    "LGBM": {
        "class_weight": [None, "balanced"]
    },
    "XGB": {
        
    }
}

## Classification

In [20]:
dataset = DEPRESJON_PREFIX  # DEPRESJON_PREFIX or PSYKOSE_PREFIX
y_filename = "depresjon_y.csv" if dataset == DEPRESJON_PREFIX else "psykose_y.csv"

In [21]:
# Assuming PROCESSED_DATA_DIR and y_filename are already defined
datasets = {}

# Load datasets
for part in ["full_24h"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()

# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

# Sprawdzamy kształt danych po usunięciu NaN
#print(datasets['full_24h'].shape)
#print(y.shape)

In [23]:
print("Wyniki dla depresjon")
for part in ["full_24h"]:
    print(f"PART: {part}")
    
    X = datasets[part]

    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        for train_idx, test_idx in folds.split(X, y):
            #print(train_idx, test_idx)
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="accuracy",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)
            
            clf = grid_search.best_estimator_
            y_pred  = clf.predict(X_test)
            metrics = calculate_metrics(y_test, y_pred)
            #print(metrics)
            test_scores.append(metrics)
        
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

Wyniki dla depresjon
PART: full_24h
  LR
    accuracy: 0.5491 +- 0.0989
    balanced_accuracy: 0.5089 +- 0.0121
    f1: 0.2151 +- 0.2635
    precision: 0.7540 +- 0.3013
    recall: 0.3568 +- 0.4369
    specificity: 0.6611 +- 0.4152
    ROC_AUC: 0.5089 +- 0.0121
    MCC: 0.0249 +- 0.0334

  SVM
    accuracy: 0.6256 +- 0.0096
    balanced_accuracy: 0.4998 +- 0.0057
    f1: 0.0204 +- 0.0185
    precision: 0.7500 +- 0.3162
    recall: 0.0108 +- 0.0101
    specificity: 0.9888 +- 0.0187
    ROC_AUC: 0.4998 +- 0.0057
    MCC: 0.0136 +- 0.0472

  RF
    accuracy: 0.6095 +- 0.0288
    balanced_accuracy: 0.5765 +- 0.0273
    f1: 0.4591 +- 0.0404
    precision: 0.4748 +- 0.0413
    recall: 0.4483 +- 0.0575
    specificity: 0.7046 +- 0.0522
    ROC_AUC: 0.5765 +- 0.0273
    MCC: 0.1558 +- 0.0565

  LGBM
    accuracy: 0.6347 +- 0.0244
    balanced_accuracy: 0.5886 +- 0.0295
    f1: 0.4526 +- 0.0497
    precision: 0.5095 +- 0.0378
    recall: 0.4100 +- 0.0654
    specificity: 0.7672 +- 0.0336
    RO

# Classification Psykose

In [24]:
dataset = PSYKOSE_PREFIX  # DEPRESJON_PREFIX or PSYKOSE_PREFIX
y_filename = "depresjon_y.csv" if dataset == DEPRESJON_PREFIX else "psykose_y.csv"

In [25]:
# Assuming PROCESSED_DATA_DIR and y_filename are already defined
datasets = {}

# Load datasets
for part in ["full_24h"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()

# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

# Sprawdzamy kształt danych po usunięciu NaN
#print(datasets['full_24h'].shape)
#print(y.shape)

In [26]:
print("Wyniki dla psykose")
for part in ["full_24h"]:
    print(f"PART: {part}")
    
    X = datasets[part]

    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        for train_idx, test_idx in folds.split(X, y):
            #print(train_idx, test_idx)
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="accuracy",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)
            
            clf = grid_search.best_estimator_
            y_pred  = clf.predict(X_test)
            metrics = calculate_metrics(y_test, y_pred)
            #print(metrics)
            test_scores.append(metrics)
        
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

Wyniki dla psykose
PART: full_24h
  LR
    accuracy: 0.5241 +- 0.0633
    balanced_accuracy: 0.5466 +- 0.0323
    f1: 0.4254 +- 0.2133
    precision: 0.5218 +- 0.2397
    recall: 0.6286 +- 0.3154
    specificity: 0.4645 +- 0.2728
    ROC_AUC: 0.5466 +- 0.0323
    MCC: 0.0984 +- 0.0663

  SVM
    accuracy: 0.6609 +- 0.0137
    balanced_accuracy: 0.5481 +- 0.0160
    f1: 0.2230 +- 0.0465
    precision: 0.6674 +- 0.0985
    recall: 0.1348 +- 0.0314
    specificity: 0.9615 +- 0.0138
    ROC_AUC: 0.5481 +- 0.0160
    MCC: 0.1769 +- 0.0563

  RF
    accuracy: 0.6037 +- 0.0222
    balanced_accuracy: 0.5725 +- 0.0280
    f1: 0.4549 +- 0.0457
    precision: 0.4544 +- 0.0345
    recall: 0.4579 +- 0.0638
    specificity: 0.6871 +- 0.0327
    ROC_AUC: 0.5725 +- 0.0280
    MCC: 0.1447 +- 0.0550

  LGBM
    accuracy: 0.6517 +- 0.0257
    balanced_accuracy: 0.6029 +- 0.0315
    f1: 0.4670 +- 0.0532
    precision: 0.5275 +- 0.0476
    recall: 0.4241 +- 0.0741
    specificity: 0.7818 +- 0.0422
    ROC_